# 目標
数独の自動解決プログラムの作成を通して、コーディングスピードを上げる

仕様書
問題を


オブジェクト
- 盤面
    - 盤面情報を持つ
    - 数値の出入力を行う
    - 


- プレイヤー
    - 考えて戦略的に数値を入れる
    - 入力は盤面から得た情報、出力は情報入力



In [1]:
import numpy as np
import pandas as pd
import copy
from queue import LifoQueue

問題をCSVファイルから読み込む→ndarrayに変換

In [2]:
def read_question_data(address):
    data = pd.read_csv(address,header=None)
    data.fillna(0,inplace=True)#空欄を0で補間する
    q_mat = np.array(data,dtype = int)
    return q_mat

In [150]:
q_0 = read_question_data("question_01.csv")

In [3]:
class Board:
    #問題の情報を保持
    #問題を解くための情報も合わせて保持
    def __init__(self):
        self.status = np.zeros((9,9))
        self.can_input = np.full((9,9,9),1)
#         グループ情報をまとめて作成
        self.box_group = []
        self.row_group = []
        self.column_group = []
        for i in range(9):
            self.box_group.append(Group("box",i))
            self.row_group.append(Group("row",i))
            self.column_group.append(Group("column",i))
    
    def input_status(self,index,number):
        self.status[index[0],index[1]] = number
            
    def input_can_input(self,index,number):
        self.status[index[0],index[1]] = number
        
        


In [32]:
class SearchAnswer:
    def __init__(self):
        self.board = Board()
        self.board_backup = LifoQueue() #スタック型のデータ構造putとgetで出し入れする
    
    def set_question (self,question):
        self.__init__()
        for row in range(9):
            for column in range(9):
                if question[row][column] != 0:
                    input_num = question[row][column]
                    self.input_number(row,column,input_num)
    
    
#     使用予定なし
#     Number_Set = [2,3,4,5,6,7,8,9]
    
    #実装
    def search_answer_single(self):
        self.all_group_search()
        self.cell_determinate_search()
#         print(self.board.status)
        
    def search_answer(self):
        blank_cell_num = self.count_blank_cell()
        print("question  blank cell:{}".format(blank_cell_num))
        print(self.board.status)
        iteration = 0
        max_iteration = 100
        while self.finish_answer() == False:
            self.search_answer_single()
            iteration += 1
            blank_cell_num = self.count_blank_cell()
            print("iteration:{}  blank cell:{}".format(iteration,blank_cell_num))
            print(self.board.status)
            
            if iteration > max_iteration:
                print("The number of iteration is over.")
                return 
        print("Finish.")
    
    def count_blank_cell(self):
        return np.sum((self.board.status == 0))
                
    def all_group_search(self):
        b_g = self.board.box_group
        r_g = self.board.row_group
        c_g = self.board.row_group
        for i in range(9):
            self.group_search(b_g[i])
            self.group_search(r_g[i])
            self.group_search(c_g[i])
            
            
            
    def cell_determinate_search(self):
        #             他の数字が入る余地がないセルを入力

        while True:
            determinate_cell =  self.board.can_input.sum(axis = 2)==1
            if np.any(determinate_cell)==True:
                ind_mat = np.array(np.where(determinate_cell == True))
                row_ar = ind_mat[0]
                column_ar = ind_mat[1]
                for i in range(len(row_ar)):
                    input_num = np.argmax(self.board.can_input[row_ar[i]][column_ar[i]])+1
                    self.input_number(row_ar[i],column_ar[i],input_num)
#                     print("cell:{},{},{}".format(row_ar[i], column_ar[i], input_num))
                continue
            else:
                break
        
#     仮実装
    def group_search(self,group):
        
        cell_ind = group.cell_index
        can_input_num = np.zeros(9)
        for row, column in cell_ind:
            can_input_num += self.board.can_input[row][column]
            
        if np.any(can_input_num == 1):
            input_num_ar = np.where(can_input_num == 1)[0] + 1
            for input_num in input_num_ar:
                for row,column in group.cell_index:
                    if self.board.can_input[row][column][input_num-1] == 1:
                        self.input_number(row,column,input_num)
#                         print("group:{},{},{}".format(row, column, input_num))
                        break
    
    #矛盾が生じたことを検出する　未実装
    def miss_answer(self):
        pass
        
            
    
    #仮実装：前提として、不正な入力をしていないことを想定し、未記入のcellがないことをもって答えかどうかのチェックを行う。
    def finish_answer(self):
        finish_flag = np.all(self.board.status != 0)
        return finish_flag
    
    def input_number(self,row,column,input_number):
#         statusに数字を代入
        self.input_board_status(row,column,input_number)
#         代入したマスのcan_inputをすべて0にする
        self.board.can_input[row][column] = self.board.can_input[row][column]*0
#         groupを更新->can_inputも更新される
        self.update_group(self.board.row_group[row],input_number)
        self.update_group(self.board.column_group[column],input_number)
        b_ind = self._search_box_index(row,column)
        self.update_group(self.board.box_group[b_ind],input_number)
        
        
    def _search_box_index(self,row,column):
        group_index = (row//3)*3 + column//3
        
        return group_index
    
    def input_board_status(self,row,column,number):
        self.board.status[row][column] = number
        
    
    def update_group(self,group,number):
        group.number_in_group[number-1] = 1
        for row,column in group.cell_index:
            self.update_can_input(row,column,number)
    
    def update_can_input(self,row,column,number):
        self.board.can_input[row][column][number-1] = 0
    
    

In [4]:
# 重複した数字が入らない１グループを意味するもの
class Group:
    def __init__(self,group_type,group_index):
#         0-1の二値でとる。
        self.number_in_group = np.zeros(9)
        if group_type == "box":
            self.cell_index = [[(group_index//3)*3 + i//3, (group_index%3)*3 + i%3] for i in range (9) ]
        elif group_type == "row":
            self.cell_index = [[group_index, i] for i in range (9)]
        elif group_type == "column":
            self.cell_index = [[i,group_index] for i in range (9)]
        

In [34]:
# q_1 = read_question_data("question_01.csv")
# q_2 = read_question_data("question_02.csv")
q_3 = read_question_data("question/question_03.csv")

In [30]:
search = SearchAnswer()

In [35]:
search.set_question(q_3)
%time search.search_answer()

question blank cell:57
[[ 7.  0.  0.  4.  0.  1.  0.  9.  0.]
 [ 0.  9.  2.  0.  0.  0.  0.  0.  0.]
 [ 0.  5.  0.  0.  8.  0.  0.  4.  0.]
 [ 0.  0.  4.  1.  0.  0.  6.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  3.  0.  9.  7.  0.  2.  8.  0.]
 [ 1.  2.  0.  0.  0.  0.  5.  0.  0.]
 [ 0.  8.  3.  0.  0.  2.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  7.]]
iteration:1 blank cell:41
[[ 7.  6.  8.  4.  0.  1.  3.  9.  0.]
 [ 4.  9.  2.  0.  0.  0.  0.  0.  0.]
 [ 3.  5.  1.  0.  8.  9.  7.  4.  0.]
 [ 0.  7.  4.  1.  0.  0.  6.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  3.  0.  9.  7.  4.  2.  8.  1.]
 [ 1.  2.  7.  0.  0.  0.  5.  0.  0.]
 [ 0.  8.  3.  7.  0.  2.  0.  0.  0.]
 [ 0.  4.  0.  0.  0.  0.  0.  2.  7.]]
iteration:2 blank cell:39
[[ 7.  6.  8.  4.  0.  1.  3.  9.  0.]
 [ 4.  9.  2.  0.  0.  7.  0.  0.  0.]
 [ 3.  5.  1.  0.  8.  9.  7.  4.  0.]
 [ 0.  7.  4.  1.  0.  0.  6.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  7.  0.]
 [ 0.  3.  0.  9.  7.  4. 

iteration:67 blank cell:39
[[ 7.  6.  8.  4.  0.  1.  3.  9.  0.]
 [ 4.  9.  2.  0.  0.  7.  0.  0.  0.]
 [ 3.  5.  1.  0.  8.  9.  7.  4.  0.]
 [ 0.  7.  4.  1.  0.  0.  6.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  7.  0.]
 [ 0.  3.  0.  9.  7.  4.  2.  8.  1.]
 [ 1.  2.  7.  0.  0.  0.  5.  0.  0.]
 [ 0.  8.  3.  7.  0.  2.  0.  0.  0.]
 [ 0.  4.  0.  0.  0.  0.  0.  2.  7.]]
iteration:68 blank cell:39
[[ 7.  6.  8.  4.  0.  1.  3.  9.  0.]
 [ 4.  9.  2.  0.  0.  7.  0.  0.  0.]
 [ 3.  5.  1.  0.  8.  9.  7.  4.  0.]
 [ 0.  7.  4.  1.  0.  0.  6.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  7.  0.]
 [ 0.  3.  0.  9.  7.  4.  2.  8.  1.]
 [ 1.  2.  7.  0.  0.  0.  5.  0.  0.]
 [ 0.  8.  3.  7.  0.  2.  0.  0.  0.]
 [ 0.  4.  0.  0.  0.  0.  0.  2.  7.]]
iteration:69 blank cell:39
[[ 7.  6.  8.  4.  0.  1.  3.  9.  0.]
 [ 4.  9.  2.  0.  0.  7.  0.  0.  0.]
 [ 3.  5.  1.  0.  8.  9.  7.  4.  0.]
 [ 0.  7.  4.  1.  0.  0.  6.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  7.  0.]
 [ 0.  3.  0.  9.  7

In [202]:
search.__init__()